In [1]:
import pandas as pd

In [25]:
df_original = pd.read_excel('data/vcd_database.xlsx', index_col=1, header=7).iloc[:,1:]
df_original.head(10)

,管理番号,実験日,化合物名,瓶番号,ＣＡＳ\n番号,分子式,測定条件,ファイル名,グラフ
グラフ枚数,,,,,,,,,
1,23,2002-08-02,(R)-(-)-Carvone,NaN,6485-40-1,C10H14O,"CCl4, 0.21 M, 72 um pathlength",rcrv7,rcrv7
2,27,2002-08-05,"(1R,2S,5R)-(-)-Menthol",NaN,2216-51-5,C10H20O,"CCl4, 1.29 M, 72 um pathlength",lmen4,lmen4
3,34,2002-08-07,(-)-Menthyl chloride,NaN,16052-42-9,C10H18Cl,"CCl4, 1.58 M, 72 um pathlength",mmcl3,mmcl3
4,37,2002-08-07,(-)-Borneol,NaN,464-45-9,C10H18O,"CCl4, 0.82 M, 72 um pathlength",mbor3,mbor3
5,40,2002-08-08,(-)-Isoplegol,NaN,89-79-2,C10H16O,"CCl4, 1.46 M, 72 um pathlength",mipg3,mipg3
6,44,2002-08-08,(R)-(+)-Pulegone,NaN,89-82-7,C10H16O,"CCl4, 0.53 M, 72 um pathlength",rplg4,rplg4
7,47,2002-08-09,"(1S,2S,5S)-(-)-Myrtanol",NaN,53369-17-8,C10H18O,"CCl4, 1.19 M, 72 um pathlength",mmyt3,mmyt3
8,64,2002-09-04,(1R)-(-)-Fenchone,NaN,7787-20-4,C10H16O,"CCl4, 0.21 M, 72 um pathlength",rfen2,rfen2
9,69,2002-09-05,(1R)-endo-(+)-Fenchyl alchol,NaN,2217-02-9,C10H18O,"CCl4, 0.83 M, 72 um pathlength",rfea2,rfea2


In [40]:
df = pd.DataFrame()
df['name'] = df_original['化合物名'].replace(
    {'(-)-Isoplegol': '(-)-Isopulegol',
     '(1R)-endo-(+)-Fenchyl alchol': '(1R)-endo-(+)-Fenchyl alcohol',
     #'Myrtenyl chloride': '161680',
     '(R)-(+)-Methyl 3-hydroxy-2-Methylpropionate': '(R)-Methyl 3-hydroxy-2-methylpropanoate',
     #'(1S)-(+)-4-Oxomyrtenyl pivalate': '688339',
     '(S)-(-)-Methyl 3-hydroxy-2-Methylpropionate': '(S)-Methyl 3-hydroxy-2-methylpropanoate',
     '(+)-Isoplegol': '(+)-Isopulegol',})
     #'(1R)-(-)-Neomenthyl acetate': '6432249',
     #'C15': 0, # Could be 84703
     #'MLC-6247': 0)
df = df.reset_index(drop=True).set_index('name')

In [27]:
#names = df_original['化合物名'].tolist()
from pyrfume.odorants import get_cids, from_cids
cids = get_cids(df.index)

  0%|          | 0/163 [00:00<?, ?it/s]

Multiple CIDs for (-)-Menthyl chloride: [85951, 244509]
Multiple CIDs for (-)-Borneol: [1201518, 64685, 18639728, 10049, 439569, 439568, 512289, 6850744, 44630107]
Multiple CIDs for (1S,2R,5R)-(+)-Isomenthol: [45056, 1254]
Multiple CIDs for (1S)-(-)-alpha-Pinene: [440968, 12223112, 82227]
Multiple CIDs for (1R)-(-)-Myrtenal: [61130, 1201529]
Could not find Myrtenyl chloride
Could not find Nopyl chloride
Multiple CIDs for (1S)-(-)-alpha-Pinene: [440968, 12223112, 82227]
Multiple CIDs for (S)-(+)-2-Nonanol: [11094753, 12367, 11286536]
Multiple CIDs for (1S)-(+)-Menthyl acetate: [12732528, 62335, 27867]
Could not find (1S)-(+)-4-Oxomyrtenyl pivalate
RDKit ERROR: [11:42:13] ERROR: 
RDKit ERROR: [11:42:14] ERROR: 
RDKit ERROR: [11:42:14] ERROR: 
RDKit ERROR: [11:42:14] ERROR: 
RDKit ERROR: [11:42:15] ERROR: 
RDKit ERROR: [11:42:15] ERROR: 
RDKit ERROR: [11:42:16] ERROR: 
RDKit ERROR: [11:42:16] ERROR: 
RDKit ERROR: [11:42:16] ERROR: 
RDKit ERROR: [11:42:17] ERROR: 
RDKit ERROR: [11:42:17] E

In [43]:
df['CID'] = df.index.map(cids.get)
df.loc['Myrtenyl chloride', 'CID'] = 161680
df.loc['(1S)-(+)-4-Oxomyrtenyl pivalate', 'CID'] = 688339
df.loc['(1R)-(-)-Neomenthyl acetate', 'CID'] = 6432249

In [63]:
df = df[~df.index.duplicated()]

In [64]:
df[df['CID']==0]

,CID
name,
Nopyl chloride,0
C15,0
MLC-6247,0


In [65]:
info = from_cids(df['CID'])

  0%|          | 0/2 [00:00<?, ?it/s]

In [79]:
from pyrfume.odorants import canonical_smiles
joined = df.join(pd.DataFrame(info).set_index('CID'), on='CID')
joined.loc['Nopyl chloride'] = canonical_smiles('ClCCC1=CCC2CC1C2(C)C')
joined.index.name = 'old_name'
joined = joined.reset_index().set_index('CID')
joined.to_csv('vcd_molecules.csv')

In [98]:
coleman_smiles = pd.read_csv('coleman_smiles.csv', index_col=0)
coleman_smiles = set(coleman_smiles.values.ravel().tolist())

In [106]:
all_smiles = set(joined['IsomericSMILES'].dropna().values.tolist()) | set(coleman_smiles)
all_smiles = set([canonical_smiles(x) for x in all_smiles])
pd.DataFrame(index=all_smiles).to_csv('enantiomer_smiles.csv')

<hr>

In [4]:
x = {i: pd.read_excel('data/vcd_database_shadmany.xlsx', sheet_name='Chart%d'%i, index_col=0) 
     for i in range(1,163) if i not in [102]}

In [5]:
list(x)

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162]

In [6]:
pd.concat(x).unstack()['ＶＣＤ']

X Values,833.18,835.11,837.04,838.97,840.90,842.83,844.76,846.68,848.61,850.54,...,1784.00,1785.90,1787.80,1789.80,1791.70,1793.60,1795.50,1797.50,1799.40,1801.30
1,-4.42160,-0.17114,0.774900,0.11816,0.020507,-0.19042,-0.643550,0.26001,1.25630,0.19946,...,-1.83590,-2.10660,-2.22820,-2.29900,-2.44180,-2.67500,-2.93970,-3.28240,-3.11540,-2.9106
2,-0.33453,-0.22918,-0.028259,0.11920,0.173950,0.55029,0.689330,0.37365,0.10363,-0.03601,...,-1.37590,-1.50230,-1.52600,-1.50540,-1.53660,-1.55780,-1.61200,-1.70060,-1.64930,-1.6080
3,0.82022,0.90368,0.987420,1.02980,0.882170,0.86087,0.803950,0.67230,0.77002,0.74270,...,-0.64529,-0.67880,-0.73834,-0.74649,-0.71688,-0.69571,-0.73602,-0.80505,-0.84014,-0.8572
4,-5.90000,-5.10710,-3.261400,-1.37250,-0.796870,-0.78079,-1.069400,-1.27090,-1.12570,-0.83068,...,1.58720,1.52080,1.50400,1.54320,1.53950,1.53790,1.51700,1.39230,1.41140,1.4685
5,-1.09440,-1.05680,-1.009400,-0.81945,-0.732660,-0.40789,0.001892,0.29565,0.48193,0.16253,...,-1.21000,-1.28170,-1.32530,-1.30010,-1.30100,-1.33530,-1.40210,-1.43750,-1.38760,-1.3881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,9.96820,-3.21090,-4.510200,-0.84326,4.952100,7.27050,2.336400,-1.80660,-0.30615,4.29490,...,3.54630,3.38910,2.82420,4.39600,4.43550,3.12890,3.07270,2.97850,1.70260,-0.1250
159,27.41100,24.21200,16.898000,14.79600,17.809000,17.46400,15.005000,6.01950,-2.51510,-3.52000,...,3.71530,4.84270,4.03120,2.22900,3.43890,4.81490,2.03560,0.24560,2.06980,1.8007
160,13.99600,14.94000,16.284000,18.93700,18.458000,16.27000,13.448000,8.63110,8.38890,10.29300,...,-2.31540,-0.84936,0.59643,-0.50268,-1.08930,0.31103,-0.10766,-1.35620,-0.45629,-0.2478
161,-4.71330,-18.28900,-25.720000,-24.00000,-13.020000,-14.53500,-22.735000,-22.31200,-19.49100,-20.67000,...,5.10400,4.92190,4.87440,4.86340,4.38580,4.03460,4.43230,4.38550,3.91200,3.3885


In [31]:
import xlrd

In [35]:
x = xlrd.open_workbook('data/vcd_database_shadmany.xlsx')

In [42]:
list(x.keys())

['6485-40-1',
 '2216-51-5',
 '16052-42-9',
 '464-45-9',
 '89-79-2',
 '89-82-7',
 '53369-17-8',
 '7787-20-4',
 '2217-02-9',
 '25465-65-0',
 '28283-97-8',
 '19894-97-4',
 '38651-65-9',
 '18172-67-3',
 '7785-26-4',
 '35836-73-8',
 '564-94-3',
 '72936-74-4',
 0,
 '74851-17-5',
 '24047-72-1',
 '26184-62-3',
 '33758-16-6',
 '52019-78-0',
 '1565-80-6',
 '57803-73-3',
 '42072-39-9',
 '1767-46-0',
 '7540-51-4',
 '464-49-3',
 '2623-23-6',
 '2552-91-2',
 '2385-77-5',
 '6033-23-4',
 '4221-99-2',
 '27871-49-4',
 '70419-06-6',
 '33758-15-5',
 '72657-23-9',
 '21210-43-5',
 '13704-09-1',
 '14898-80-7',
 '14898-86-3',
 '5157-89-1',
 '14898-79-4',
 '113473-32-8',
 '31087-44-2',
 '14073-97-3',
 '6169-06-8',
 '26549-24-6',
 '19210-21-0',
 '5989-27-5',
 '5978-70-1',
 '6033-24-5',
 '464-48-2',
 '17392-83-5',
 '16404-54-9',
 '80657-57-4',
 '70419-07-7',
 '498-15-7',
 '5989-54-8',
 '1117-61-9',
 '18680-27-8',
 '56777-24-3',
 '22144-60-1',
 '536-59-4',
 '27779-29-9',
 '7785-70-8',
 '10606-72-1',
 '10334-26-6',

In [ ]:
vcd = pd.DataFrame(index=smiles, columns=wavelengths)
ir = pd.DataFrame(index=smiles, columns=wavelengths)

In [6]:
CIDs = pd.read_csv('data/vcd-data-shadmany.csv')

FileNotFoundError: File b'data/vcd-data-shadmany.csv' does not exist